## LOAD SOLUTION FILES

In [1]:
import os
import pickle
import pandas as pd
from attention import *
from helpers import *

### Extract OD demand and path set (X and Y)
X: OD demand, graph (link feature), path, link-path adj \
Y: path flow

In [2]:
%run helpers.py
%run attention.py

class Dataset():
    def __init__(self, size, input_dim0, input_dim1, output_dim0, output_dim1, start_from=0):
        super().__init__()
        self.path_encoded = path_encoder() # Get path encode dictionary
        self.entries = size
        self.X = torch.zeros([size, input_dim0, input_dim1], dtype=torch.float32)
        self.Y = torch.zeros([size, output_dim0, output_dim1], dtype=torch.float32)

        for i in tqdm(range(size)) :
            file_name = f"Output/5by5_Data{start_from+i}"
            x, y = generate_xy(file_name, self.path_encoded)
            self.X[i] = x
            self.Y[i] = y
    
    def __len__(self):
        return self.entries

    def __getitem__(self, idx):
        data_point = self.X[idx]
        data_label = self.Y[idx]
        return data_point, data_label
    

In [ ]:
# %run helpers.py

# Load first 1000 sample data
train_size = 10
train_dataset = Dataset(train_size, 625, 1165, 625, 3)
# this code create batch data, shape batch_size x seq_length x dim
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)

### Test Encoder

In [2]:
# print(torch.cuda.is_available())
%run helpers.py

size = 30
sequence_leng = 625
input_dim = 1164
output_dim = 3
path_encoded = path_encoder()

X = torch.zeros([size, sequence_leng, input_dim], dtype=torch.float32)
Y = torch.zeros([size, sequence_leng, output_dim], dtype=torch.float32)
network = []
for i in tqdm(range(size)) :
    file_name = f"Output/5by5_Data{i}"
    x, y = generate_xy(file_name, path_encoded)
    X[i] = x 
    Y[i] = y 

    # file = open(file_name, "rb")
    # stat = pickle.load(file)
    # file.close()
    # net = stat["data"]["network"]
    # Graph = normalize_tensor(get_graphTensor(net))
    # Graph = get_graphTensor(net)
    # network.append(Graph)

    

  0%|          | 0/30 [00:00<?, ?it/s]

In [4]:
%run attention.py

# X = torch.flatten(X, start_dim=1)
print("input shape: ", X.shape)

x2 = Encoder(input_dim=input_dim, d_model=2048, N=2, heads=8, dropout=0.1)(X)
print("Encoder output shape: ", x2.shape)

input shape:  torch.Size([30, 625, 1164])
Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention
Finish encoder
Encoder output shape:  torch.Size([30, 625, 2048])


In [23]:
df = pd.DataFrame(X[0].numpy())
df.iloc[:10, :6]


,0,1,2,3,4,5
0,0.000000,0.000000,0.041667,NaN,NaN,NaN
1,0.012658,0.041667,0.000000,NaN,NaN,NaN
2,0.025316,0.000000,0.208333,NaN,NaN,NaN
3,0.037975,0.208333,0.000000,NaN,NaN,NaN
4,0.050633,0.041667,0.083333,NaN,NaN,NaN
5,0.063291,0.083333,0.041667,NaN,NaN,NaN
6,0.075949,0.041667,0.250000,NaN,NaN,NaN
7,0.088608,0.250000,0.041667,NaN,NaN,NaN
8,0.101266,0.083333,0.125000,NaN,NaN,NaN
9,0.113924,0.125000,0.083333,NaN,NaN,NaN


### Test Decoder

In [7]:
%run attention.py
decoder = Decoder(output_dim, 2048, 2, 8, 0.1)
x3 = decoder(Y, x2)
x3.shape

Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention


torch.Size([30, 625, 2048])

### Test Transformer

In [3]:
%run attention.py

transformer = Transformer(input_dim=input_dim, output_dim=output_dim, d_model=2048, N=2,heads=8, dropout=0.1)
out = transformer(X, Y)
out.shape

Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention
Finish encoder
Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention
Finish calculate attention score
Finish Multihead attention


torch.Size([625, 3])

In [4]:
df = pd.DataFrame(out[0].detach().numpy())
df.head()

,0
0,-0.497437
1,0.029807
2,0.186898
